<a href="https://colab.research.google.com/github/yskuchi/wf_denoising/blob/master/denoising2D3out_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Waveform denoising 'denoising2D3out_transfer'
Author: Yusuke Uchiyama

A denoising & deconvolution convolutional autoencoder with Tensorflow2.x
applied to **a set of** waveform data.  
Outputs are (0) estimated noise waveform, (1) estimated signal waveform, (2) deconvolved impulse signal (= cluster timing). 
See [Bitbucket repository](https://bitbucket.org/meg_ilc_tokyo/wf_denoising/src/master/) or 
[GitHub repository](https://github.com/yskuchi/wf_denoising)

Training start with trained model for denoising part (transfer learning).

Noise from data is added to MC signal data.
You need datasets of signal and noise, separately, in pickle format.

## Environment
As of 2022 Aug, tested with the following:

* Google Colab
* CPU, GPU, or TPU (experimental)
* Python 3.7
* TensorFlow 2.8.2
* Comet ML 3.31


Note: If you are running this in a colab notebook, we recommend you enable a free GPU by going:
> Runtime   →   Change runtime type   →   Hardware Accelerator: GPU

## Setting

### Comet ML

In [ ]:
! pip install typing-extensions comet-ml
! pip install typing-extensions
#! [ ! -z "$COLAB_GPU" ] && pip install typing-extensions==3.7.4 comet-ml

In [ ]:
# import comet_ml in the top of your file
from comet_ml import Experiment

### Other packages

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import json
import gc

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from tensorflow.keras.layers import Dense, Reshape, Flatten, Concatenate, Add, Subtract
from tensorflow.keras.layers import Dropout, Activation, BatchNormalization
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint

print ('Python version: ' + str(sys.version_info))
print ('TensorFlow version: ' + str(tf.__version__))

### GPU
To use GPU on Google colab, specify GPU in runtime type. 

In [ ]:
# check GPU
tf.test.gpu_device_name()
!echo $COLAB_GPU

### TPU
To use TPU on Google colab, it is not enough to specify TPU in runtime type.
See "Setup TPU".

In [ ]:
# check TPU
!echo $COLAB_TPU_ADDR

### Parameters

In [ ]:
# arg
load_weights = False
plot_data = True 

import matplotlib
if not plot_data:
    matplotlib.use("Agg") # this is necessary when using plt without display (batch)
import matplotlib.pyplot as plt

In [ ]:
# infer signal or noise?
extract_signal = False

# Number of channels (if 2, two-end signals are dealt with channel)
nchannels = 2

# Number of waveforms packed in an input data (= Hight of 2D data)
height = 8

if extract_signal:
  filename = f'denoising2D{height}3out_tf2'
else:
  filename = f'noiseextraction2D{height}ch3out_tf2'

# Waveform has 1024 sample-points
npoints = 512 #1024 # 256 # number of sample-points to be used (= Width of 2D data)
scale = 1 #5
offset = 0.000 #0.05 # 50 mV

signal_scales = [1]  # multiple scales -> data augmentation

fine_tuning = True  # flag to apply fine-tuning

#signal_dataset_file = 'wf11600.pkl.gz'
##noise_dataset_file  = 'wf328469.pkl' #2018
#noise_dataset_file  = 'wf356990.pkl.gz' #2020
signal_dataset_file = 'wf_cdch13000.pkl.gz'
noise_dataset_file  = 'wf402042.pkl.gz' #2021
cluster_dataset_file = 'cls1st_cdch13000.pkl.gz' 

# base model
base_model_filename = 'noiseextraction2D8ch2out_tf2_20220824_2.h5'
base_model_weights_filename = 'noiseextraction2D8ch2out_tf2_20220824_2_weights.h5'

#### Hyper-parameters

In [ ]:
nkernels = [128, 128, 64,   64, 128, 128]

kernel_size = [[1,7], [3,7], [3,5], [1,7], [1,7], [1,5]]
strides =[[1,1], [1,1], [1,1]]
pooling_size = [[1,2],[2,2],[2,2], [1,2],[1,2], [1,2]]

# basic hyper-parameters
params = {
    'optimizer':   'adam',
    'loss':        'mse', #'mae', #'msle', #'binary_crossentropy', 
    'loss_weights': [0.15, 0.05, 0.8],
    'metrics':     ['mae', 'mse'],
    'epochs':      1,#50, #150, # 20,
    'batch_size':  1024, #512, #256,
    'steps_per_execution': 100, # default 1, for TPU larger value may help
}
if extract_signal:
  params['loss'] = 'msle'

# additional parameters
params2 = {
    'loss_type':           params['loss'],
    'conv_activation':     'relu',
    'output_activation':   ['linear', 'sigmoid'],
    'signal_dataset_file': signal_dataset_file,
    'noise_dataset_file':  noise_dataset_file,
    'File name':           filename + '.ipynb',
    'nchannels':           nchannels,
    'nrows':               height,
    'npoints':             npoints,
    'batch_size':          params['batch_size'],
    'scale':               scale,
    'offset':              offset,
    'nkernels':            nkernels,
    'kernel_size':         kernel_size,
    'skip_connection':     [True, True, True, True, True, True], # skip connections for UNet-like structure
    'nsublayers':          1,
    'batch_norm':          [0,0,0, 0,1,0,0,1,0]
}

## Prepare datasets
On Google Colb, data is loaded via Google Drive.
Files are supposed to be in `/content/drive/My Drive/ML/data`.

### Mount Google Drive

In [ ]:
## The following doesn't work anymore for Drive in other account (as of 2022 Aug).
## To mount Drive in other account, do the three steps below
#from google.colab import drive
#drive.mount('/content/drive')
#data_dir = '/content/drive/My Drive/ML/data/'
#output_dir = '/content/drive/My Drive/ML/results/'

In [ ]:
## Here is a trick to mount Drive in other account
## First, run this column and click the URL shown in the error message.
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

In [ ]:
!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

In [ ]:
data_dir = '/content/drive/MyDrive/ML/data/'
output_dir = '/content/drive/MyDrive/ML/results/'

### Load pickle files

In [ ]:
x_signal = pd.read_pickle(data_dir+signal_dataset_file).to_numpy()
x_noise = pd.read_pickle(data_dir+noise_dataset_file ).to_numpy()
x_cluster = pd.read_pickle(data_dir+cluster_dataset_file ).to_numpy()

In [ ]:
# Data-augmentation with different scales
print(x_signal.shape)
x_signal_ori = x_signal
x_cluster_ori = x_cluster
for signal_scale in signal_scales:
  if signal_scale != 1.0:
    x_scaled = x_signal_ori * signal_scale
    x_signal = np.concatenate([x_signal, x_scaled])
    x_cluster_scaled = x_cluster_ori * signal_scale
    x_cluster = np.concatenate([x_cluster, x_cluster_scaled])
print(x_signal.shape)
del x_signal_ori
del x_cluster_ori

# For noise data, repeat if fewer than signal data
x_noise_ori = x_noise
while True:
  if len(x_noise) > len(x_signal):
    break
  x_noise = np.concatenate([x_noise, x_noise_ori])
del x_noise_ori
gc.collect()

In [ ]:
nsamples = min(len(x_signal), len(x_noise)) 
nsamples = int(nsamples / (nchannels * height)) * nchannels * height

print(f'signal samples:{len(x_signal)}, noise samples:{len(x_noise)}, nsamples: {nsamples}')

if extract_signal:
  print('Extract signal')
  x_tobe_removed = x_noise[0:nsamples]
  x_tobe_extracted = x_signal[0:nsamples]
else:
  print('Extract noise')
  x_tobe_removed = x_signal[0:nsamples]
  x_tobe_extracted = x_noise[0:nsamples]
x_tobe_compared = x_cluster[0:nsamples]

### Shape data in appropriate format with adding noise

In [ ]:
x_tobe_removed = x_tobe_removed.astype('float32')
x_tobe_removed = x_tobe_removed.T[-npoints:].T # keep last npoints
x_tobe_extracted = x_tobe_extracted.astype('float32')
x_tobe_extracted = x_tobe_extracted.T[-npoints:].T # keep last npoints
x_tobe_compared = x_tobe_compared.astype('float32')
x_tobe_compared = x_tobe_compared.T[-npoints:].T # keep last npoints

# Add noise
x_train_noisy = x_tobe_removed + x_tobe_extracted

# Adjust scale and offset of waveforms
x_tobe_removed *= scale
x_tobe_removed += offset * scale
x_tobe_extracted *= scale # scale
x_tobe_extracted += offset * scale;
x_train_noisy *= scale # scale
x_train_noisy += offset * scale; # add offset
x_tobe_compared *= scale

## Values in [0,1]
#x_tobe_extracted = np.clip(x_tobe_extracted, 0, 1);
#x_train_noisy = np.clip(x_train_noisy, 0, 1);

# To match the input shape for Conv2D with n channels
ninputs = int(len(x_tobe_removed) / (nchannels * height))
x_tobe_removed = np.reshape(x_tobe_removed, (ninputs, height, nchannels, npoints)).transpose(0,1,3,2)
x_tobe_extracted = np.reshape(x_tobe_extracted, (ninputs, height, nchannels, npoints)).transpose(0,1,3,2)
x_train_noisy = np.reshape(x_train_noisy, (ninputs, height, nchannels, npoints)).transpose(0,1,3,2)
x_tobe_compared = np.reshape(x_tobe_compared, (ninputs, height, nchannels, npoints)).transpose(0,1,3,2)

print(x_tobe_removed.shape)

## Model

In [ ]:
# Add the following code anywhere in your machine learning file
# api_key and workspace are supposed to be set in .comet.config file,
# otherwise set here like Experiment(api_key="AAAXXX", workspace = "yyy", project_name="zzz")
# experiment = Experiment(project_name="wf_denoising")
experiment = Experiment(api_key="gBJn86Y1oAYKM2oxaoY0oV4Af", workspace="yskuchi", project_name="wf_denoising")

In [ ]:
experiment.log_parameters(params2)

### Setup TPU
This part seems tf version dependent and may be changed.

In [ ]:
if 'COLAB_TPU_ADDR' in os.environ:
  tpu_grpc_url = "grpc://"+os.environ["COLAB_TPU_ADDR"]
  tpu_cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_grpc_url)
  tf.config.experimental_connect_to_cluster(tpu_cluster_resolver) # TF2.0の場合、ここを追加
  tf.tpu.experimental.initialize_tpu_system(tpu_cluster_resolver) # TF2.0の場合、今後experimentialが取れる可能性がある    
  #strategy = tf.distribute.experimental.TPUStrategy(tpu_cluster_resolver)  # ここも同様
  strategy = tf.distribute.TPUStrategy(tpu_cluster_resolver)  # experimentalいらなくなった
  #model = tf.distribute.tpu.keras_to_tpu_model(model, strategy=strategy)

### Build model with functional API

In [ ]:
def build_model():

  # denoising part
  base_model = load_model(output_dir + base_model_filename)
  base_model.summary()
  input_img = base_model.input
  decoded1 = base_model.get_layer('decoded1').output
  decoded2 = base_model.get_layer('decoded2').output

  # deconvolution part
  conv1 = Conv2D(params2['nkernels'][0], (kernel_size[3][0], kernel_size[3][1]), strides=(strides[0][0], strides[0][1]), padding='same', name='conv1_deco')(decoded2)
  if params2['batch_norm'][3]:
    conv1 = BatchNormalization()(conv1)
  #conv1 = Dropout(0.2)(conv1)
  conv1 = Activation(params2['conv_activation'])(conv1)
  for i in range(1, params2['nsublayers']):
    conv1 = Conv2D(params2['nkernels'][0], (kernel_size[3][0], kernel_size[3][1]), strides=(1,1), padding='same', activation=params2['conv_activation'], name=f'conv1_{i}_deco')(conv1)
  pool1 = MaxPooling2D((pooling_size[3][0], pooling_size[3][1]), padding='same', name='max_pooling2d_1_deco')(conv1)
  conv2 = Conv2D(params2['nkernels'][1], (kernel_size[4][0], kernel_size[4][1]), strides=(strides[1][0], strides[1][1]), padding='same', name='conv2_deco')(pool1)
  if params2['batch_norm'][4]:
    conv2 = BatchNormalization()(conv2)
  #conv2 = Dropout(0.2)(conv2)
  conv2 = Activation(params2['conv_activation'])(conv2)
  for i in range(1, params2['nsublayers']):
    conv2 = Conv2D(params2['nkernels'][1], (kernel_size[4][0], kernel_size[4][1]), strides=(1,1), padding='same', activation=params2['conv_activation'], name=f'conv2_{i}_deco')(conv2)
  pool2 = MaxPooling2D((pooling_size[4][0], pooling_size[4][1]), padding='same', name='max_pooling2d_2_deco')(conv2)
  conv3 = Conv2D(params2['nkernels'][2], (kernel_size[5][0], kernel_size[5][1]), strides=(strides[2][0], strides[2][1]), padding='same', name='conv3_deco')(pool2)
  if params2['batch_norm'][5]:
    conv3 = BatchNormalization()(conv3)
  #conv3 = Dropout(0.2)(conv3)
  conv3 = Activation(params2['conv_activation'])(conv3)
  for i in range(1, params2['nsublayers']):
    conv3 = Conv2D(params2['nkernels'][2], (kernel_size[5][0], kernel_size[5][1]), strides=(1,1), padding='same', activation=params2['conv_activation'], name=f'conv3_{i}_deco')(conv3)
  encoded = MaxPooling2D((pooling_size[5][0], pooling_size[5][1]), padding='same')(conv3)

  conv4 = Conv2D(params2['nkernels'][3], (kernel_size[5][0], kernel_size[5][1]), strides=(1, 1), padding='same', name='conv4_deco')(encoded)
  #conv4 = Conv2DTranspose(params2['nkernels'][3], (kernel_size[5][0], kernel_size[5][1]), strides=(1, 1), padding='same')(encoded)
  if params2['batch_norm'][6]:
    conv4 = BatchNormalization()(conv4)
  #conv4 = Dropout(0.2)(conv4)
  conv4 = Activation(params2['conv_activation'])(conv4)
  for i in range(1, params2['nsublayers']):
    conv4 = Conv2D(params2['nkernels'][3], (kernel_size[5][0], kernel_size[5][1]), strides=(1,1), padding='same', activation=params2['conv_activation'], name=f'conv4_{i}_deco')(conv4)
  conv4 = UpSampling2D((pooling_size[5][0], pooling_size[5][1]), name='up_sampling2d_1_deco')(conv4)
  if params2['skip_connection'][5]:
    conv4 = Concatenate(name='concat1_deco')([conv4, conv3])
  conv5 = Conv2DTranspose(params2['nkernels'][4], (kernel_size[4][0], kernel_size[4][1]), strides=(strides[2][0], strides[2][1]), padding='same', name='convT5_deco')(conv4)
  if params2['batch_norm'][7]:
    conv5 = BatchNormalization()(conv5)
  #conv5 = Dropout(0.2)(conv5)
  conv5 = Activation(params2['conv_activation'])(conv5)
  for i in range(1, params2['nsublayers']):
    conv5 = Conv2D(params2['nkernels'][4], (kernel_size[4][0], kernel_size[4][1]), strides=(1,1), padding='same', activation=params2['conv_activation'], name=f'convT5_{i}_deco')(conv5)
  conv5 = UpSampling2D((pooling_size[4][0], pooling_size[4][1]), name='up_sampling2d_2_deco')(conv5)
  if params2['skip_connection'][4]:
    conv5 = Concatenate(name='concat2_deco')([conv5, conv2])
  conv6 = Conv2DTranspose(params2['nkernels'][5], (kernel_size[3][0], kernel_size[3][1]), strides=(strides[1][0], strides[1][1]), padding='same', name='convT6_deco')(conv5)
  if params2['batch_norm'][8]:
    conv6 = BatchNormalization()(conv6)
  #conv6 = Dropout(0.2)(conv6)
  conv6 = Activation(params2['conv_activation'])(conv6)
  for i in range(1, params2['nsublayers']):
    conv6 = Conv2D(params2['nkernels'][5], (kernel_size[3][0], kernel_size[3][1]), strides=(1,1), padding='same', activation=params2['conv_activation'], name=f'convT6_{i}_deco')(conv6)
  conv6 = UpSampling2D((pooling_size[3][0], pooling_size[3][1]), name='up_sampling2d_3_deco')(conv6)
  if params2['skip_connection'][3]:
    conv6 = Concatenate(name='concat3_deco')([conv6, conv1])
  for i in range(params2['nsublayers'] - 1):
    conv6 = Conv2D(nchannels, (1, 3), padding='same', activation=params2['conv_activation'], name='conv6_deco')(conv6)
  decoded3 = Conv2DTranspose(nchannels, (1, 3), strides=(strides[0][0], strides[0][1]), padding='same', activation=params2['output_activation'][1], name='decoded3')(conv6)
  

  autoencoder = Model(inputs=input_img, outputs=[decoded1, decoded2, decoded3])

  for layer in base_model.layers[:-1]: # 18 to fix all layers
    layer.trainable = False

  if params['steps_per_execution'] == 1:
    autoencoder.compile(optimizer=params['optimizer'], 
                        loss={'decoded1': params['loss'], 'decoded2': 'msle', 'decoded3': 'binary_crossentropy'},
                        loss_weights={'decoded1': params['loss_weights'][0], 'decoded2': params['loss_weights'][1], 'decoded3': params['loss_weights'][2]}, 
                        metrics=params['metrics'])
  else:
    autoencoder.compile(optimizer=params['optimizer'], 
                        loss={'decoded1': params['loss'], 'decoded2': 'msle', 'decoded3': 'binary_crossentropy'},
                        loss_weights={'decoded1': params['loss_weights'][0], 'decoded2': params['loss_weights'][1], 'decoded3': params['loss_weights'][2]}, 
                        metrics=params['metrics'],
                        steps_per_execution=params['steps_per_execution'])

  autoencoder.summary()

  #autoencoder.load_weights(f'{output_dir}{base_model_weights_filename}', by_name=True)
  
  return autoencoder

In [ ]:
try:
  strategy
  with strategy.scope():
    autoencoder = build_model()
except NameError:
  autoencoder = build_model()

## Fit

On Google Colb, the results (trained model) are saved in Google Drive. Files are supposed to be in /content/drive/My Drive/ML/results.

In [ ]:
history=[]
if not load_weights:

    # Callback for model checkpoints
    checkpoint = ModelCheckpoint(
        filepath = output_dir + filename + "-{epoch:02d}.h5",
        save_best_only=True,
        save_weight_only=False)
    
    # 'labels' are the pictures themselves
    hist = autoencoder.fit(x_train_noisy,
                           {'decoded1': x_tobe_extracted, 'decoded2': x_tobe_removed, 'decoded3': x_tobe_compared},
                           epochs=params['epochs'],
                           batch_size=params['batch_size'],
                           shuffle=True,
                           validation_split=0.1)#,
                           #callbacks=[checkpoint])


    # Save history
    with open(output_dir + filename + '_hist.json', 'w') as f:
        json.dump(hist.history, f)
    history = hist.history
        
    # Save the weights
    autoencoder.save_weights(output_dir + filename + '_weights.h5')
else:
    # Load weights
    autoencoder.load_weights(f'{output_dir}{filename}_weights.h5')

    # Load history
    with open(f'{output_dir}{filename}_hist.json', 'r') as f:
        history = json.load(f)

## Save the model

In [ ]:
# as a keras model (a single .h5 file)
autoencoder.save(output_dir + filename + '.h5', include_optimizer=False)
# as a TF model (a directory) <- doesn't work with TPU ???
# autoencoder.save(output_dir + filename)

## Convert to ONNX
#! pip install onnxmltools 
#! pip install tf2onnx
#import onnxmltools
#print ('ONNXMLTools version:' + str(onnxmltools.__version__))
#onnx_model = onnxmltools.convert_keras(autoencoder)
## Save as protobuf <- doesn't work with TPU ???
#onnxmltools.utils.save_model(onnx_model, output_dir + filename + '.onnx')

# Plot training history 
#plt.plot(history['loss'], linewidth=3, label='train')
#plt.plot(history['val_loss'], linewidth=3, label='valid')
plt.plot(history['decoded1_mae'], linewidth=3, label='train')
plt.plot(history['val_decoded1_mae'], linewidth=3, label='valid')
plt.grid()
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
if params['metrics'] == 'mse':
  plt.ylim(1e-6 * scale, 0.1e-3 * scale) #mse
else:
  plt.ylim(0.5e-3 * scale, 0.5e-2 * scale) #mae
plt.show()

## Fine tuning
Repeat training for a few layers 

In [ ]:
if fine_tuning:
  for layer in autoencoder.layers[:-1]: 
    layer.trainable = False
  for layer in autoencoder.layers[13:18]:
    layer.trainable = True
  if params['steps_per_execution'] == 1:
    autoencoder.compile(optimizer=RMSprop(learning_rate=1e-5), 
                        loss={'decoded1': params['loss'], 'decoded2': 'msle', 'decoded3': 'binary_crossentropy'},
                        loss_weights={'decoded1': params['loss_weights'][0], 'decoded2': params['loss_weights'][1], 'decoded3': params['loss_weights'][2]}, 
                        metrics=params['metrics'])
  else:
    autoencoder.compile(optimizer=RMSprop(learning_rate=1e-5), 
                        loss={'decoded1': params['loss'], 'decoded2': 'msle', 'decoded3': 'binary_crossentropy'},
                        loss_weights={'decoded1': params['loss_weights'][0], 'decoded2': params['loss_weights'][1], 'decoded3': params['loss_weights'][2]}, 
                        metrics=params['metrics'],
                        steps_per_execution=params['steps_per_execution'])

  autoencoder.summary()

In [ ]:
if fine_tuning:
  history=[]
  hist = autoencoder.fit(x_train_noisy,
                         {'decoded1': x_tobe_extracted, 'decoded2': x_tobe_removed, 'decoded3': x_tobe_compared},
                          epochs=params['epochs'],
                          batch_size=params['batch_size'],
                          shuffle=True,
                          validation_split=0.1)

  # Save history
  with open(output_dir + filename + '_hist.json', 'w') as f:
      json.dump(hist.history, f)
  history = hist.history
        
  # Save the weights
  autoencoder.save_weights(output_dir + filename + '_weights.h5')

In [ ]:
if fine_tuning:
  # as a keras model (a single .h5 file)
  autoencoder.save(output_dir + filename + '.h5', include_optimizer=False)
  # as a TF model (a directory) <- doesn't work with TPU ???
  # autoencoder.save(output_dir + filename)

  ## Convert to ONNX
  #! pip install onnxmltools 
  #! pip install tf2onnx
  #import onnxmltools
  #print ('ONNXMLTools version:' + str(onnxmltools.__version__))
  #onnx_model = onnxmltools.convert_keras(autoencoder)
  ## Save as protobuf <- doesn't work with TPU ???
  #onnxmltools.utils.save_model(onnx_model, output_dir + filename + '.onnx')

  # Plot training history 
  #plt.plot(history['loss'], linewidth=3, label='train')
  #plt.plot(history['val_loss'], linewidth=3, label='valid')
  plt.plot(history['decoded1_mae'], linewidth=3, label='train')
  plt.plot(history['val_decoded1_mae'], linewidth=3, label='valid')
  plt.grid()
  plt.legend()
  plt.xlabel('epoch')
  plt.ylabel('loss')
  if params['metrics'] == 'mse':
    plt.ylim(1e-6 * scale, 0.1e-3 * scale) #mse
  else:
    plt.ylim(0.5e-3 * scale, 0.5e-2 * scale) #mae
  plt.show()

## Test

In [ ]:
x_test_tobe_removed = x_tobe_removed[0:]
x_test_tobe_extracted = x_tobe_extracted[0:]
x_test_noisy = x_train_noisy[0:]
x_test_tobe_compared = x_tobe_compared[0:]
print(x_test_noisy.shape)

decoded_imgs, decoded_imgs2, decoded_imgs3 = autoencoder.predict(x_test_noisy)
decoded_imgs = decoded_imgs[0:len(x_test_noisy)]
decoded_imgs3 = decoded_imgs3[0:len(x_test_noisy)]
print(decoded_imgs.shape)

# revert scale and offset
x_test_tobe_removed -= scale * offset
x_test_tobe_removed /= scale
x_test_tobe_extracted -= scale * offset
x_test_tobe_extracted /= scale
x_test_noisy -= scale * offset
x_test_noisy /= scale
x_test_tobe_compared /= scale
decoded_imgs -= scale * offset
decoded_imgs /= scale
decoded_imgs3 /= scale

print(x_test_tobe_extracted.shape)
print(x_test_noisy.shape)

x_test_tobe_removed = x_test_tobe_removed.transpose(0, 1, 3, 2)
x_test_tobe_removed = np.reshape(x_test_tobe_removed, (len(x_test_tobe_removed) * height * nchannels, npoints))
x_test_tobe_extracted = x_test_tobe_extracted.transpose(0, 1, 3, 2)
x_test_tobe_extracted = np.reshape(x_test_tobe_extracted, (len(x_test_tobe_extracted) * height * nchannels, npoints))
x_test_noisy = x_test_noisy.transpose(0,1,3,2)
x_test_noisy = np.reshape(x_test_noisy, (len(x_test_noisy) * height * nchannels, npoints))
x_test_tobe_compared = x_test_tobe_compared.transpose(0, 1, 3, 2)
x_test_tobe_compared = np.reshape(x_test_tobe_compared, (len(x_test_tobe_compared) * height * nchannels, npoints))
decoded_imgs = decoded_imgs.transpose(0,1,3,2)
decoded_imgs = np.reshape(decoded_imgs, (len(decoded_imgs) * height * nchannels, npoints))
decoded_imgs3 = decoded_imgs3.transpose(0,1,3,2)
decoded_imgs3 = np.reshape(decoded_imgs3, (len(decoded_imgs3) * height * nchannels, npoints))

x_residual = x_test_tobe_extracted - decoded_imgs
x_decoded_signal = decoded_imgs
if not extract_signal:
  x_decoded_signal = x_test_noisy - decoded_imgs

In [ ]:
# How many waveforms to be displayed
n = 2
start = 1760
fig = plt.figure(figsize=(20, 6 * n))
j = 0
for i in range(start, start + n):
  ax = fig.add_subplot(n * 2, 1, j+1)
  ax.plot(x_test_noisy[i], label="noisy", color='gray')
  if extract_signal:
    ax.plot(x_test_tobe_extracted[i], label="signal", color='green')
  else:
    ax.plot(x_test_tobe_extracted[i], label="noise", color='green')
    ax.plot(x_test_tobe_removed[i], label="signal")
    ax.plot(x_decoded_signal[i], label="extracted signal")
  ax.plot(decoded_imgs[i], label="decoded", color='magenta')
  ax.plot(x_residual[i], label="residual")
  ax.legend()
  j += 1

  ax = fig.add_subplot(n * 2, 1, j+1)
  ax.plot(x_test_tobe_compared[i], label="signal")
  ax.plot(decoded_imgs3[i], label="decoded", color='magenta')
  ax.legend()
  j += 1

In [ ]:
# Send this plot to comet
experiment.log_figure(figure=fig)

In [ ]:
experiment.end()

In [ ]:
if plot_data:
    plt.show()